Vamos a scrapear el sitio de LATAM para averiguar datos de vuelos en funcion al origen y destino, fecha y cabina. La informacion que esperamos obtener de cada vuelo es:

- Precio(s) disponibles
- Horas de salida y llegada (duracion)
- Informacion de las escalas

In [10]:
import requests
from bs4 import BeautifulSoup

In [11]:
url = 'https://www.latam.com/es_mx/apps/personas/booking?fecha1_dia=12&fecha1_anomes=2020-10&fecha2_dia=14&fecha2_anomes=2020-10&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=11/10/2020&vuelos_fecha_regreso_ddmmaaaa=11/10/2020&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [12]:
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(url, headers=agent)
#r = requests.get(url)

In [13]:
r.status_code

200

In [14]:
s = BeautifulSoup(r.text, 'lxml')

In [15]:
print(s.prettify())
#<div>
#    Para una mejor experiencia en LATAM.com, habilite JavaScript
#    en su navegador
#</div>

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Selecciona tus vuelos | LATAM Airlines
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s.latamstatic.com/etc.clientlibs/latamApp/clientlibs/static/resources/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="personas_bs3_latam_mu" name="layout"/>
  <meta content="LATAM.com" name="description"/>
  <meta content="LATAM.com" name="keywords"/>
  <script src="https://www.pontosmultiplus.com.br/cestaautenticacao/app/multiplus-sdk.js" type="text/javascript">
  </script>
  <meta content="7F7CFDC9CA48BEDC62079EA62FB597F9" name="sessionId" scheme="tracker"/>
  <meta content="latam" name="matchedScenario" scheme="tracker"/>
  <meta content="personas_bs3_latam_mu" name="template" scheme="tracker"/>
  <meta content="null" na

# Selenium

Selenium es una herramienta que nos permitira controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la pagina. Para esto necesitamos el modulo webdriver.

In [16]:
from selenium import webdriver

Paso 1: Instanciar un driver del navegador

In [206]:
options = webdriver.FirefoxOptions()
options.add_argument("-private")
driver = webdriver.Firefox(options=options)

Paso 2: Hacer que el navegador cargue la pagina web

In [207]:
driver.get(url)

Pas 3: Extraer informacion de la pagina

In [181]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="06dd29c0-b214-4ad0-a389-7396acd38c16", element="9124f99a-dc5a-4ad3-8573-bd7170b77b68")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="06dd29c0-b214-4ad0-a389-7396acd38c16", element="858b647f-3463-49a1-9eb1-4d2cbf11fd8f")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="06dd29c0-b214-4ad0-a389-7396acd38c16", element="bb294247-0557-49af-8a31-c27489d869d8")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="06dd29c0-b214-4ad0-a389-7396acd38c16", element="fd0ba340-0a40-4604-8478-fe3b5252e14c")>]

In [185]:
vuelo = vuelos[0]

In [186]:
vuelo #session= La sesion hace referencia al elemento del navegador que tenemos automatizado.

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="06dd29c0-b214-4ad0-a389-7396acd38c16", element="9124f99a-dc5a-4ad3-8573-bd7170b77b68")>

In [184]:
#Hora de salida
hour_departure = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
print(f'Hora de salida: {hour_departure}')

Hora de salida: 12:43


In [120]:
#Hora de llegada
hour_arrival = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
print(f'Hora de llegada: {hour_arrival}')

Hora de llegada: 14:40


In [121]:
#Tiempo del viaje
duration = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
duration_travel = duration.replace('PT', '').lower()
print(f'Tiempo de viaje: {duration_travel}')

Tiempo de viaje: 20h57m


In [122]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [123]:
boton_escalas.click()

In [124]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
segmentos #En este ejemplo arroja 3 elementos por que son 3 escalas para este vuelo

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="1b5be151-f795-4711-b0b0-017ed8d2bf96")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="5f2cf0e8-8852-4f9b-b1d7-da5c81ad0eac")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="d819a890-79c2-4c7b-975d-8a3d750a563f")>]

In [125]:
escalas = len(segmentos) - 1
escalas

2

In [126]:
segmento = segmentos[0]

In [127]:
segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//div[@class="sc-bwCtUz iybVbT"]/abbr').get_attribute('title')

'EZE'

In [128]:
segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'12:43'

In [129]:
#Obtener el destino de cada escala

In [130]:
#Obtener la hora de llegada de cada escala

In [131]:
#Obtener la duracion de cada vuelo

In [132]:
#Obtener la duracion de cada escala

In [133]:
#Obtener numero de vuelo

In [134]:
#Obtener modelo del avion

In [135]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

In [202]:
vuelo.click()

InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [140]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [141]:
tarifas

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="a712db7d-eba1-4163-8418-cb447e7c48e5")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="0ea6f3c4-6225-41bd-8d40-10be6baf9f32")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="01b0b924-fc5a-417d-b3d5-6c1b301348ea", element="b857e47f-df99-410e-919d-3823122ea3bd")>]

In [ ]:
#Encontra la moneda y valor de cada vuelo`

In [218]:
def obtener_precios(vuelo):
    '''
    Función quye retorna una lista de diccionarios con las distintas tarifas
    '''
    #tarifas=vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    tarifas=vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    #print(tarifas)
    precios=[]
    for tarifa in tarifas:
        nombre=tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda=tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor =tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
        dict_tarifa={'nombre':nombre,'moneda':moneda,'valor':valor}
        precios.append(dict_tarifa)
        print(dict_tarifa)
    return precios

In [217]:
vuelo = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelo = vuelos[0]

obtener_precios(vuelo)

[]


[]

In [165]:
def obtener_datos_escalas(vuelo):
    '''
    Función  que retorna una lista de diccionarios 
    con las información de las escalas de cada vuelo
    '''
    segmentos=vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg bOhNnO"]/div[@class="sc-cLQEGU hyoued"]')
    info_escalas = []
    for segmento in segmentos:
        #origen=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//abbr[@class="sc-hrWEMg hlCkST"]')[0].text
        origen = segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//div[@class="sc-bwCtUz iybVbT"]/abbr').get_attribute('title')
        #hra_salida=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//time')[0].get_attribute('datetime')
        hra_salida = segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')
        #destino=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//abbr[@class="sc-hrWEMg hlCkST"]')[1].text
        #hra_llegad=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//time')[1].get_attribute('datetime')
        #duracion=segmento.find_element_by_xpath('.//span[@class="sc-cmthru ipcOEH"]//time').get_attribute('datetime')
        #num_vuelo=segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS dhxqHN"]//b').text
        #mod_avion=segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS dhxqHN"]//span[@class="sc-gzOgki uTyOl"]').text
        
        data_dict={
            'origen':origen,
            'hr_salida':hra_salida #,
            #'destino':destino,
            #'hra-llegada':hra_salida,
            #'duracion':duracion,
            #'num_vuelo':num_vuelo,
            #'mod_avion':mod_avion
        }
        info_escalas.append(data_dict)
        
    return info_escalas

In [187]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos
vuelo = vuelos[0]
obtener_tiempos(vuelo)

{'hora_salida': '12:43', 'hora_llegada': '14:40', 'duracion': 'PT20H57M'}

In [190]:
def obtener_tiempos(vuelo):
    '''
    Funcion que retorna un diccionario con los horarios de salida y llegada de cada vuelo, incluyendo la duracion.
    Nota: La duracion del vuelo no es la hora de llegada - la hora de salida porque puede haber diferencia de 
    horarios entre el origen y el destino.
    '''
    #Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    #Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    #Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    tiempos = {'hora_salida':salida,'hora_llegada':llegada,'duracion':duracion}
    return tiempos

In [195]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    #print(f'Se encontraron {len[vuelos]} vuelos.')
    print('Iniciando el scraping...')
    info = []
    for vuelo in vuelos:
        #Obtenemos los tiempos generales de cada vuelo
        tiempos = obtener_tiempos(vuelo)
        #Clickeamos sobre el boton de escalas
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        escalas = obtener_datos_escalas(vuelo)
        #Cerrarmos el modal
        driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()
        #Clickeamos el vuelo para ver los precios
        vuelo.click()
        precios = obtener_precios(vuelo)
        vuelo.click()
        info.append({'precios':precios, 'tiempos':tiempos, 'escalas':escalas})
    return info

In [222]:
import time

In [228]:
#Modulos que nos ayudaran a que termine de cargar la pagina para que inicie el scraping
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [230]:
options = webdriver.FirefoxOptions()
options.add_argument("-private")
driver = webdriver.Firefox(options=options)
driver.get(url)
#Introducir una demora
#time.sleep(10)
delay = 15
try:
    #Introducir demora inteligente
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print("La pagina termino de cargar")
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La pagina tardo demasiado en cargar.')
    info_vuelos = []
driver.close()

La pagina termino de cargar
Iniciando el scraping...
{'nombre': 'LIGHT', 'moneda': 'US$', 'valor': '455'}
{'nombre': 'PLUS', 'moneda': 'US$', 'valor': '525'}
{'nombre': 'TOP', 'moneda': 'US$', 'valor': '603'}
{'nombre': 'LIGHT', 'moneda': 'US$', 'valor': '455'}
{'nombre': 'PLUS', 'moneda': 'US$', 'valor': '525'}
{'nombre': 'TOP', 'moneda': 'US$', 'valor': '603'}
{'nombre': 'LIGHT', 'moneda': 'US$', 'valor': '532'}
{'nombre': 'PLUS', 'moneda': 'US$', 'valor': '616'}
{'nombre': 'TOP', 'moneda': 'US$', 'valor': '710'}
{'nombre': 'LIGHT', 'moneda': 'US$', 'valor': '555'}
{'nombre': 'PLUS', 'moneda': 'US$', 'valor': '633'}
{'nombre': 'TOP', 'moneda': 'US$', 'valor': '717'}


In [ ]:
info_vuelos

In [221]:
driver.close()

Paso 4: Cerrar el navegador

In [194]:
#driver = webdriver.Firefox()
driver.close()